# Домашнее задание. Нейросетевая классификация текстов

В этом домашнем задании вам предстоит самостоятельно решить задачу классификации текстов на основе семинарского кода. Мы будем использовать датасет [ag_news](https://paperswithcode.com/dataset/ag-news). Это датасет для классификации новостей на 4 темы: "World", "Sports", "Business", "Sci/Tech".

Установим модуль datasets, чтобы нам проще было работать с данными.

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


In [ ]:
3+7

10

Импорт необходимых библиотек

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List, Callable
import string

import seaborn
seaborn.set(palette='summer')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных
Для вашего удобства, мы привели код обработки датасета в ноутбуке. Ваша задача --- обучить модель, которая получит максимальное возможное качество на тестовой части.

In [ ]:
# Загрузим датасет
dataset = datasets.load_dataset('ag_news')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Как и в семинаре, выполним следующие шаги:
* Составим словарь
* Создадим класс WordDataset
* Выделим обучающую и тестовую часть, создадим DataLoader-ы.

In [ ]:
words = Counter()

for example in tqdm(dataset['train']['text']):
    # Приводим к нижнему регистру и убираем пунктуацию
    prccessed_text = example.lower().translate(
        str.maketrans('', '', string.punctuation))

    for word in word_tokenize(prccessed_text):
        words[word] += 1


vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
counter_threshold = 25

for char, cnt in words.items():
    if cnt > counter_threshold:
        vocab.add(char)

print(f'Размер словаря: {len(vocab)}')

word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

  0%|          | 0/120000 [00:00<?, ?it/s]

Размер словаря: 11842


In [ ]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        processed_text = self.data[idx]['text'].lower().translate(
            str.maketrans('', '', string.punctuation))
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) for word in word_tokenize(processed_text)
            ]
        tokenized_sentence += [self.eos_id]

        train_sample = {
            "text": tokenized_sentence,
            "label": self.data[idx]['label']
        }

        return train_sample

    def __len__(self) -> int:
        return len(self.data)


def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>'], max_len=256) -> torch.Tensor:
    seq_lens = [len(x['text']) for x in input_batch]
    max_seq_len = min(max(seq_lens), max_len)

    new_batch = []
    for sequence in input_batch:
        sequence['text'] = sequence['text'][:max_seq_len]
        for _ in range(max_seq_len - len(sequence['text'])):
            sequence['text'].append(pad_id)

        new_batch.append(sequence['text'])

    sequences = torch.LongTensor(new_batch).to(device)
    labels = torch.LongTensor([x['label'] for x in input_batch]).to(device)

    new_batch = {
        'input_ids': sequences,
        'label': labels
    }

    return new_batch

In [ ]:
train_dataset = WordDataset(dataset['train'])

np.random.seed(42)
torch.manual_seed(42)

idx = np.random.choice(np.arange(len(dataset['test'])), 5000)
eval_dataset = WordDataset(dataset['test'].select(idx))

batch_size = 32
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, shuffle=False, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Постановка задачи
Ваша задача -- получить максимальное возможное accuracy на `eval_dataloader`. Ниже приведена функция, которую вам необходимо запустить для обученной модели, чтобы вычислить качество её работы.

In [ ]:
def evaluate(model, eval_dataloader) -> float:
    """
    Calculate accuracy on validation dataloader.
    """

    predictions = []
    target = []
    with torch.no_grad():
        for batch in eval_dataloader:
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))
            target.append(batch['label'])

    predictions = torch.cat(predictions)
    target = torch.cat(target)
    accuracy = (predictions == target).float().mean().item()

    return accuracy

## Ход работы
Оценка за домашнее задание складывается из четырех частей:
### Запуск базовой модели с семинара на новом датасете (1 балл)
На семинаре мы создали модель, которая дает на нашей задаче довольно высокое качество. Ваша цель --- обучить ее и вычислить `score`, который затем можно будет использовать в качестве бейзлайна.

В модели появится одно важное изменение: количество классов теперь равно не 2, а 4. Обратите на это внимание и найдите, что в коде создания модели нужно модифицировать, чтобы учесть это различие.

### Проведение экспериментов по улучшению модели (2 балла за каждый эксперимент)
Чтобы улучшить качество базовой модели, можно попробовать различные идеи экспериментов. Каждый выполненный эксперимент будет оцениваться в 2 балла. Для получения полного балла за этот пункт вам необходимо выполнить по крайней мере 2 эксперимента. Не расстраивайтесь, если какой-то эксперимент не дал вам прироста к качеству: он все равно зачтется, если выполнен корректно.

Вот несколько идей экспериментов:
* **Модель RNN**. Попробуйте другие нейросетевые модели --- LSTM и GRU. Мы советуем обратить внимание на [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html), так как интерфейс этого класса ничем не отличается от обычной Vanilla RNN, которую мы использовали на семинаре.
* **Увеличение количества рекуррентных слоев модели**. Это можно сделать с помощью параметра `num_layers` в классе `nn.RNN`. В такой модели выходы первой RNN передаются в качестве входов второй RNN и так далее.
* **Изменение архитектуры после применения RNN**. В базовой модели используется агрегация со всех эмбеддингов. Возможно, вы захотите конкатенировать результат агрегации и эмбеддинг с последнего токена.
* **Подбор гиперпараметров и обучение до сходимости**. Возможно, для получения более высокого качества просто необходимо увеличить количество эпох обучения нейросети, а также попробовать различные гиперпараметры: размер словаря, `dropout_rate`, `hidden_dim`.

Обратите внимание, что главное правило проведения экспериментов --- необходимо совершать одно архитектурное изменение в одном эксперименте. Если вы совершите несколько изменений, то будет неясно, какое именно из изменений дало прирост к качеству.

### Получение высокого качества (3 балла)
В конце вашей работы вы должны указать, какая из моделей дала лучший результат, и вывести качество, которое дает лучшая модель, с помощью функции `evaluate`. Ваша модель будет оцениваться по метрике `accuracy` следующим образом:
* $accuracy < 0.9$ --- 0 баллов;
* $0.9 \leqslant accuracy < 0.91$ --- 1 балл;
* $0.91 \leqslant accuracy < 0.915$ --- 2 балла;
* $0.915 \leqslant accuracy$ --- 3 балла.

### Оформление отчета (2 балла)
В конце работы подробно опишите все проведенные эксперименты.
* Укажите, какие из экспериментов принесли улучшение, а какие --- нет.
* Проанализируйте графики сходимости моделей в проведенных экспериментах. Являются ли колебания качества обученных моделей существенными в зависимости от эпохи обучения, или же сходимость стабильная?
* Укажите, какая модель получилась оптимальной.

Желаем удачи!

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
num_epoch = 5
eval_steps = len(train_dataloader) // 2


losses_type = {}
acc_type = {}

for aggregation_type in ['max', 'mean']:
    print(f"Starting training for {aggregation_type}")
    losses = []
    acc = []

    model = CharLM(
        hidden_dim=256, vocab_size=len(vocab), aggregation_type=aggregation_type).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()
        for i, batch in enumerate(tqdm(train_dataloader, desc=f'Training epoch {epoch}:')):
            optimizer.zero_grad()
            logits = model(batch['input_ids'])
            loss = criterion(logits, batch['label'])
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            if i % eval_steps == 0:
                model.eval()
                acc.append(evaluate(model, eval_dataloader))
                model.train()

        losses.append(sum(epoch_losses) / len(epoch_losses))

    losses_type[aggregation_type] = losses
    acc_type[aggregation_type] = acc

Starting training for max


Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

...

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

In [ ]:
acc_type

{'max': [0.24699999392032623,
  0.8823999762535095,
  0.8931999802589417,
  0.8759999871253967,
  0.8998000025749207,
  0.8949999809265137,
  0.9016000032424927,
  0.9009999632835388,
  0.9021999835968018,
  0.8987999558448792],
 'mean': [0.26260000467300415,
  0.8765999674797058,
  0.8799999952316284,
  0.8981999754905701,
  0.8899999856948853,
  0.9057999849319458,
  0.9045999646186829,
  0.9085999727249146,
  0.902999997138977,
  0.9070000052452087]}

Baseline best cases:
- max = 0.9076
- mean = 0.904

# Experiments

- GRU/LSTM
- ReLU for linear
- Different optimizer (AdamW)
- Gradient schedular

Для начала создадим функцию train, чтобы каждый раз просто её вызывать, вместо переписывания цикла снова и снова

In [ ]:
def train(model: Callable,
          train_loader: DataLoader,
          eval_loader: DataLoader,
          num_epochs: int = 5,
          optimizer: torch.optim.Optimizer = None,
          criterion = None,
          scheduler: Callable = None,
          device: str = 'cpu'):

    eval_steps = len(train_dataloader) // 2

    # в качестве значений по умолчанию используются параметры из самого ноутбука
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters())

    if criterion is None:
        criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])

    acc = []
    losses = []

    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()

        for i, batch in enumerate(tqdm(train_loader, desc=f'Training epoch {epoch}:')):
            optimizer.zero_grad()
            logits = model(batch['input_ids'])
            loss = criterion(logits, batch['label'])
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

            if i % eval_steps == 0:
                model.eval()
                acc.append(evaluate(model, eval_loader))
                model.train()

        # если у нас есть scheduler, то мы его применем
        if not scheduler is None:
            scheduler.step()

    losses.append(sum(epoch_losses) / len(epoch_losses))

    return acc, losses

Прогоним модель ещё раз, чтобы убедиться что:
- Функция правильно написана
- Дальнейшее обучение смысла не имеет

In [ ]:
train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.9081999659538269,
  0.9035999774932861,
  0.9031999707221985,
  0.9021999835968018,
  0.9031999707221985,
  0.9111999869346619,
  0.9013999700546265,
  0.902999997138977,
  0.9007999897003174,
  0.8998000025749207],
 [0.09390584162473678])

## Меньше шаг - 0.9104

Итак, наша модель упёрлась в её потолок немногим больше 0.9. Как мы видели, она достаточно быстро пришла к потолку. Попробуем уменьшить lr, может это поможет модели за счёт меньших шагов лучше попасть в минимум.


In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.2467999905347824,
  0.8447999954223633,
  0.8797999620437622,
  0.8895999789237976,
  0.8989999890327454,
  0.8991999626159668,
  0.9041999578475952,
  0.8944000005722046,
  0.8947999477386475,
  0.8939999938011169],
 [0.17528322613363465])

In [ ]:
3*4

12

## GRU - 0.9001

Итак, сейчас мы попробуем заменить RNN слой на GRU. Так механизм запоминания может помочь нам сохранить и передать какие-то важные параметры дальше. Для этого просто меняем nn.RNN на nn.GRU, параметры можно оставить те же

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.2457999885082245,
  0.8267999887466431,
  0.8657999634742737,
  0.8739999532699585,
  0.8863999843597412,
  0.8921999931335449,
  0.8903999924659729,
  0.8937999606132507,
  0.901199996471405,
  0.8965999484062195],
 [0.20135441896642248])

## GRU-BiDir - 0.9116

Несмотря на то, что GRU не дала нам прироста, мы можем попробовать улучшить модель, "пустив память" в двух направлениях с помощью флага bidirectional в nn.GRU

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_dim*2, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.2565999925136566,
  0.8737999796867371,
  0.9023999571800232,
  0.9053999781608582,
  0.9088000059127808,
  0.9157999753952026,
  0.911799967288971,
  0.9091999530792236,
  0.9106000065803528,
  0.9079999923706055],
 [0.06720301840649917])

## GRU-BiDir 2 layers - 0.9114

Отлично, давайте добавим ещё одни GRU слой, чтобы попробовать запомнить и передать какие-нибудь нужные признаки ещё раз, сделав своеобразный "отбор". Для этого поменяем параметр num_layers в GRU, сделав его равным 2.

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.linear = nn.Linear(hidden_dim*2, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.24039998650550842,
  0.8772000074386597,
  0.892799973487854,
  0.9034000039100647,
  0.9149999618530273,
  0.9075999855995178,
  0.9156000018119812,
  0.9109999537467957,
  0.9041999578475952,
  0.9125999808311462],
 [0.0720919440822055])

## ReLU - 0.9116

Теперь давайте добавим функцию активации для линейного слоя. Куда его ставить - перед дропаутом линейного слоя

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.linear = nn.Linear(hidden_dim*2, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.relu(self.linear(self.non_lin(output))))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.2533999979496002,
  0.8755999803543091,
  0.8899999856948853,
  0.9007999897003174,
  0.9109999537467957,
  0.9041999578475952,
  0.9115999937057495,
  0.9025999903678894,
  0.9063999652862549,
  0.9079999923706055],
 [0.0831055839508772])

## Weight Decay - 0.9142


Weight Decay представляет собой L2 регуляризацию, встроенную в оптимизатор AdamW.

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=15e-3)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.26659998297691345,
  0.8795999884605408,
  0.896399974822998,
  0.8967999815940857,
  0.9034000039100647,
  0.9099999666213989,
  0.914199948310852,
  0.9081999659538269,
  0.9125999808311462,
  0.9065999984741211],
 [0.08561258925627917])

## Plateau Scheduler -  0.9135

Эксперементы с расписанием

In [ ]:
def train(model: Callable,
          train_loader: DataLoader,
          eval_loader: DataLoader,
          num_epochs: int = 5,
          optimizer: torch.optim.Optimizer = None,
          criterion = None,
          scheduler: Callable = None,
          device: str = 'cpu'):

    eval_steps = len(train_dataloader) // 2

    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters())

    if criterion is None:
        criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])

    acc = []
    losses = []

    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()

        for i, batch in enumerate(tqdm(train_loader, desc=f'Training epoch {epoch}:')):
            optimizer.zero_grad()
            logits = model(batch['input_ids'])
            loss = criterion(logits, batch['label'])
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

            if i % eval_steps == 0:
                model.eval()
                acc.append(evaluate(model, eval_loader))
                model.train()
        if not scheduler is None:
            scheduler.step(acc[-1])

    losses.append(sum(epoch_losses) / len(epoch_losses))

    return acc, losses

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=15e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      scheduler=scheduler,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.26319998502731323,
  0.8781999945640564,
  0.8915999531745911,
  0.9025999903678894,
  0.9088000059127808,
  0.9025999903678894,
  0.9113999605178833,
  0.9035999774932861,
  0.9138000011444092,
  0.913599967956543],
 [0.08782698124554009])

## StepLR Scheduler - 0.8802

Давайте попробуем другой планировщик, который будет каждую эпоху меньшать lr в 0.5 раз. Плюс к этому давайте сделаем lr побольше, так как он будет сильнее уменьшаться

In [ ]:
def train(model: Callable,
          train_loader: DataLoader,
          eval_loader: DataLoader,
          num_epochs: int = 5,
          optimizer: torch.optim.Optimizer = None,
          criterion = None,
          scheduler: Callable = None,
          device: str = 'cpu'):

    eval_steps = len(train_dataloader) // 2

    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters())

    if criterion is None:
        criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])

    acc = []
    losses = []

    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()

        for i, batch in enumerate(tqdm(train_loader, desc=f'Training epoch {epoch}:')):
            optimizer.zero_grad()
            logits = model(batch['input_ids'])
            loss = criterion(logits, batch['label'])
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

            if i % eval_steps == 0:
                model.eval()
                acc.append(evaluate(model, eval_loader))
                model.train()
        if not scheduler is None:
            scheduler.step()

    losses.append(sum(epoch_losses) / len(epoch_losses))

    return acc, losses

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3, weight_decay=15e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      scheduler=scheduler,
      device=device)

Training epoch 0::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 1::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 2::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 3::   0%|          | 0/3750 [00:00<?, ?it/s]

Training epoch 4::   0%|          | 0/3750 [00:00<?, ?it/s]

([0.257999986410141,
  0.884399950504303,
  0.8339999914169312,
  0.8560000061988831,
  0.8531999588012695,
  0.8705999851226807,
  0.8761999607086182,
  0.8751999735832214,
  0.8801999688148499,
  0.8799999952316284],
 [0.3022846476962169])

## LSTM - 0.9176

Теперь попробуем вместо GRU использовать LSTM


In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.linear = nn.Linear(hidden_dim*2, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.relu(self.linear(self.non_lin(output))))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3, weight_decay=15e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=5,
      optimizer=optimizer,
      criterion=criterion,
      scheduler=scheduler,
      device=device)

NameError: name 'CharLM' is not defined

В этом месет у меня упал ноутбук, притом что время было к сдаче(

## LSTM, GRU - 0.9157

Порог в 0.915 с трудом но заборен.

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.gru = nn.GRU(hidden_dim*2, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.linear = nn.Linear(hidden_dim*2, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1)


        output = self.dropout(self.relu(self.linear(self.non_lin(output))))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3, weight_decay=15e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=10,
      optimizer=optimizer,
      criterion=criterion,
      scheduler=scheduler,
      device=device)

## Less penalty 0.9178 (лучший результат)

Достичь супер результата не получилось, однако можно поэксперементировать с lr

In [ ]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab), aggregation_type = 'mean').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3, weight_decay=15e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)

train(model=model,
      train_loader=train_dataloader,
      eval_loader=eval_dataloader,
      num_epochs=10,
      optimizer=optimizer,
      criterion=criterion,
      scheduler=scheduler,
      device=device)

# Итог



На улучшения скора повлияли:
1) ReLU для линейного слоя
2) Двунаправленные LSTM/GRU
3) Уменьшение lr (сразу или постпенное согласно расписанию) в некоторых случаях

Однако стоит понимать, что их необходимо применять не всегда. Нам пришлось подобрать величину уменьшения нашего lr с помощью StepLR, а также использовать как основу LSTM (+ GRU), вместо RNN/GRU. В целом LSTM дал нам наибольший прирост после GRU, что делает его более подходящим для нашей задачи (хоть и его вычисление стоит дороже).

